## This notebook shows how to generate the lattice path dataset using Sage.

For documentation of the Sage functions we used, see: 
- https://doc.sagemath.org/html/en/reference/diophantine_approximation/sage/rings/continued_fraction.html
- https://doc.sagemath.org/html/en/reference/combinat/sage/combinat/dyck_word.html

Author: Helen Jenne

In [1]:
from collections import Counter, defaultdict
import pickle
import numpy as np
import random
import csv
import operator

### Step 1: create dictionaries D_m and D_l:
D_m.items() are of the form ( m,  (p, m)  ), where m is a matching number and p is a lattice path with that matching number
D_l.items() are of the form ( (D, q), (p, l) ), where (D, q) is the (discriminant, denominator) representation of a Lagrange number, l is the float representation of that Lagrange number, and p is a lattice path with that Lagrange number. 

In [2]:
def find_differences(p, q):
    return [q[i]- p[i] for i in range(len(p))]

def to_continued_fraction(p):
    C = []
    w = list(p)
    for i in range(len(w)-1):
        if w[i] == w[i+1]:
            C += [1, 1]
        else:
            C.append(2)
    return continued_fraction(C)

def to_periodic_continued_fraction_as_list(p):
    C = []
    w = list(p)
    for i in range(len(w)-1):
        if w[i] == w[i+1]:
            C += [1, 1]
        else:
            C.append(2)
    return [2] + C

def find_p_and_q(C):
    CF = continued_fraction(C)
    return CF.numerator(len(C)), CF.denominator(len(C))

def find_r_and_s(C):
    CF = continued_fraction(C[:-1])
    return CF.numerator(len(C)), CF.denominator(len(C))

def find_discriminant_and_q(C):
    p, q = find_p_and_q(C)
    r, s = find_r_and_s(C)
    D = (p-s)**2 + 4*r*q
    return D, q

def find_Lagrange_number(p):
    """
    Given a lattice path p, returns two representations of the Lagrange number:
    as a float and as a (discriminant, denominator) pair. 
    """
    C = to_periodic_continued_fraction_as_list(p)
    L = []
    D_q = []
    for i in range(len(C)):
        shift = np.roll(C, -i)
        D, q = find_discriminant_and_q(list(shift))
        L.append( np.sqrt(D)/q )
        D_q.append( (D, q) )
    lagrange_number = max(L)
    ind_of_lagrange_number = np.argmax(L)
    return lagrange_number, D_q[ind_of_lagrange_number]

In [3]:
#check example from https://www.samuelfhopkins.com/OPAC/files/proceedings/schiffler.pdf
p = [1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0]
C = to_continued_fraction(p)
print(C)
print(C.numerator(len(C)))
print(C.denominator(len(C)))

[1; 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2]
40199
23549


In [4]:
#check example from https://www.samuelfhopkins.com/OPAC/files/proceedings/schiffler.pdf
p = [1, 1,1, 0, 1,   0, 1, 1,   0, 0, 1, 0, 0]
C = to_continued_fraction(p)
print(C)
print(C.numerator(len(C)))
print(C.denominator(len(C)))

[1; 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2]
40199
24653


In [5]:
p = [1, 1, 0, 1, 0]
C = to_continued_fraction(p)
print(C)
C.numerator(len(C))

[1; 1, 2, 2, 2]


29

#### Make valid lattice paths

In [6]:
def compute_num_ones(p):
    num_ones = []
    for i in range(len(p)):
        num_ones.append(sum(p[:i+1]))
    return num_ones

def compute_num_zeros(p):
    p = [(num +1) %2 for num in p]
    num_ones = []
    for i in range(len(p)):
        num_ones.append(sum(p[:i+1]))
    return num_ones

def below_diagonal(num_ones, num_zeros):
    for i in range(len(num_ones)):
        if num_ones[i] - num_zeros[i] < 1:
            return False
    return True

In [ ]:
if not os.path.exists("./poset_data_final/"):
    os.makedirs("./poset_data_final/", mode = 0o770)

In [220]:
for a in range(10, 14):
    print(f"a: {a}")
    b = a-1
    print(f"b: {b}")
    
    #These paths stay below the diagonal y = x
    dyck_paths = DyckWords(a, b)
    print(f"number of paths that stay below the diagonal y = x: {len(dyck_paths)}")

    paths = []
    for i, p in enumerate(dyck_paths):
        num_ones =  compute_num_ones(p)
        num_zeros = compute_num_zeros(p)
        if below_diagonal(num_ones, num_zeros):
            paths.append(p)
            

    pickle.dump(paths, open(f"./poset_data_final/paths_{a}_{b}.pkl", "wb" ) )
    
    print(f"There are {len(paths)} allowed paths")

    D_m = defaultdict(list) #key matching number, value (path, matching number)
    M = dict() #key path, value matching number

    for i, p in enumerate(paths):
        c= to_continued_fraction(p)
        m = c.numerator(len(c))

        D_m[m].append( (list(p), m) )
        M[tuple(p)] = m

    pickle.dump(D_m, open(f"./poset_data_final/Dm_{a}_{b}.pkl", "wb" ) )
    pickle.dump(M, open(f"./poset_data_final/matching_numbers_{a}_{b}.pkl", "wb" ) )

    print(f"There are {len(D_m)} different matching numbers")

    D_l = defaultdict(list) #key (D, Q), value (path, lagrange number)
    L = dict()
    for i, p in enumerate(paths):
        lag_num, (D, Q) = find_Lagrange_number(p)
        D_l[(D, Q)].append( (list(p), lag_num ) ) 
        L[tuple(p)] = lag_num

        if i % 10000 == 0:
            print(i)

    pickle.dump(D_l, open(f"./poset_data_final/Dl_{a}_{b}.pkl", "wb" ) )
    pickle.dump(L, open(f"./poset_data_final/lagrange_numbers_{a}_{b}.pkl", "wb" ) )

    print(f"There are {len(D_l)} different Lagrange numbers")

a: 6
b: 5
number of paths that stay below the diagonal y = x: 132
There are 42 allowed paths
There are 42 different matching numbers
0
There are 26 different Lagrange numbers
a: 7
b: 6
number of paths that stay below the diagonal y = x: 429
There are 132 allowed paths
There are 131 different matching numbers
0
There are 76 different Lagrange numbers
a: 8
b: 7
number of paths that stay below the diagonal y = x: 1430
There are 429 allowed paths
There are 427 different matching numbers
0
There are 232 different Lagrange numbers
a: 9
b: 8
number of paths that stay below the diagonal y = x: 4862
There are 1430 allowed paths
There are 1430 different matching numbers
0
There are 750 different Lagrange numbers
a: 10
b: 9
number of paths that stay below the diagonal y = x: 16796
There are 4862 allowed paths
There are 4859 different matching numbers
0
There are 2494 different Lagrange numbers
a: 11
b: 10
number of paths that stay below the diagonal y = x: 58786
There are 16796 allowed paths
Ther


KeyboardInterrupt



In [59]:
#If D_m, D_l, L, M have already been saved
#a = 13
#b = 12
#paths = pickle.load( open(f"./poset_data_final/paths_{a}_{b}.pkl", "rb" ) )

#D_m = pickle.load( open(f"./poset_data_final/Dm_{a}_{b}.pkl", "rb" ) )
#D_l = pickle.load( open(f"./poset_data_final/Dl_{a}_{b}.pkl", "rb" ) )
#L = pickle.load( open(f"./poset_data_final/lagrange_numbers_{a}_{b}.pkl", "rb" ) )
#M = pickle.load( open(f"./poset_data_final/matching_numbers_{a}_{b}.pkl", "rb" ) )

## Step 2: Make covering dict

A covering dictionary is has keys lattice paths, and values are lists of covers of that lattice paths

In [50]:
def make_covering_dict(D, K):
    """
    Parameters
    ----------
    D: dict
        keys are either matching numbers (as integers) or Lagrange numbers (as (D, q) representation)
        values are (paths, n) where n is the matching number or the float representation of the Lagrange number
    K: list
        A sorted list of Lagrange number or matching numbers
    
    Returns
    -------
    covering_dict: dict
        A dictionary where keys are lattice paths (represented as binary tuples), and
        values are lists of lattice paths that cover the corresponding key path.
        
    """
    covering_dict = defaultdict(list)

    for i in range(len(K)-1):
        for (p, l_num) in D[K[i]]:
            for (q, l_num) in D[K[i+1]]:
                covering_dict[tuple(p)].append(q)
    return covering_dict

In [51]:
D_l_items_list = list(D_l.items())
lagrange_numbers = sorted(D_l_items_list, key = operator.itemgetter(1, 0, -1))
matching_numbers = sorted(D_m.keys())
K = [x[0] for x in lagrange_numbers]
covering_dict = dict()
covering_dict['lagrange'] = make_covering_dict(D_l, K)
covering_dict['matching'] = make_covering_dict(D_m, matching_numbers)                                               

### Step 3: We create a dataset of tuples (p, q, diff): p is a lattice path, q is (one of its) covers, and diff is the sequence q[i] - p[i]

In [52]:
def check_for_cover_in_both(p, covering_dict):
    exists_cover_in_both = False
    for q in covering_dict['lagrange'][p]:
        if q in covering_dict['matching'][p]:
            exists_cover_in_both = True
    return exists_cover_in_both

#### We do a train/test split on the lattice paths:

In [53]:
np.random.seed(32)
split = 0.8
ds_size = int(len(paths))

random_idx = list(range(len(paths)))
np.random.shuffle(random_idx)

In [54]:
paths_train = np.array(paths)[random_idx][:math.ceil(ds_size*split)]
paths_test = np.array(paths)[random_idx][math.ceil(ds_size*split):]

In [55]:
def make_pairs(P, covering_dict):

    covers_in_both = []
    covers_in_lagrange_only = []
    covers_in_matching_only = []

    for p in P:
        p = tuple(p)
        if p != tuple([1]*a + [0]*b): #This is the maximal element and will not have a cover
            exists_cover_in_both = check_for_cover_in_both(p, covering_dict)

            if exists_cover_in_both == False:
                for q in covering_dict['lagrange'][p]:
                    covers_in_lagrange_only.append((list(p), q ))

                for q in covering_dict['matching'][p]:
                    if q in covering_dict['lagrange'][p]:
                        print("ERROR THIS SHOULD NOT HAPPEN")
                    else:
                        covers_in_matching_only.append((list(p), q ))
           # else:
              #  covers_in_both.append((list(p), q, find_differences(p, q) ))

    return covers_in_lagrange_only, covers_in_matching_only#, covers_in_both


In [56]:
covers_in_lagrange_only_train, covers_in_matching_only_train = make_pairs(paths_train, covering_dict)

In [57]:
covers_in_lagrange_only_test, covers_in_matching_only_test = make_pairs(paths_test, covering_dict)

In [58]:
for (L, poset_type, train_or_test) in [(covers_in_lagrange_only_train, "lagrange", "train"), 
                                  (covers_in_lagrange_only_test, "lagrange", "test"), 
                                  (covers_in_matching_only_train, "matching", "train"), 
                                  (covers_in_matching_only_test, "matching", "test")]:
    arr = []
    for row in L:
        flattened_row = [item for sublist in row for item in sublist]
        flattened_row.insert(a+b, ";")
        arr.append(flattened_row)

    np.savetxt(f"./poset_data_final/{poset_type}_covers_{train_or_test}_{a}_{b}.csv", arr, fmt="%s")